[Table of Contents](http://nbviewer.ipython.org/github/rlabbe/Kalman-and-Bayesian-Filters-in-Python/blob/master/table_of_contents.ipynb)

# Installation, Python, NumPy, and FilterPy

In [1]:
#format the book
%matplotlib inline
from __future__ import division, print_function
from book_format import load_style
load_style()

This book is written in Jupyter Notebook, a browser based interactive Python environment that mixes Python, text, and math. I choose it because of the interactive features - I found Kalman filtering nearly impossible to learn until I started working in an interactive environment. It is difficult to form an intuition about many of the parameters until you can change them and immediately see the output. An interactive environment also allows you to play 'what if' scenarios. "What if I set $\mathbf{Q}$ to zero?" It is trivial to find out with Jupyter Notebook.

Another reason I choose it is because most textbooks leaves many things opaque. For example, there might be a beautiful plot next to some pseudocode. That plot was produced by software, but software that is not available to the reader. I want everything that went into producing this book to be available to you. How do you plot a covariance ellipse? You won't know if you read most books. With Jupyter Notebook all you have to do is look at the source code.

Even if you choose to read the book online you will want Python and the SciPy stack installed so that you can write your own Kalman filters. There are many different ways to install these libraries, and I cannot cover them all, but I will cover a few typical scenarios.

## Installing the SciPy Stack

This book requires IPython, Jupyter, NumPy, SciPy, SymPy, and Matplotlib. The SciPy stack of NumPy, SciPy, and Matplotlib depends on third party Fortran and C code, and is not trivial to install from source code. The SciPy website strongly urges using a pre-built installation, and I concur with this advice.

I use the Anaconda distribution from Continuum Analytics. This is an excellent distribution that combines all of the packages listed above, plus many others. Installation is very straightforward, and it can be done alongside other Python installations you might already have on your machine. It is free to use. You may download it from here: http://continuum.io/downloads I strongly recommend using the latest Python 3 version that they provide.

There are other choices for installing the SciPy stack. You can find instructions here: http://scipy.org/install.html

Many Linux distributions come with these packages preinstalled. However, they are often somewhat dated and they will need to be updated as the book depends on recent versions of all. Updating a specific Linux installation is beyond the scope of this book. An advantage of the Anaconda distribution is that it does not modify your local Python installation, so you can install it and not break your linux distribution. 

## Installing FilterPy

FilterPy is a Python library that implements all of the filters used in this book, and quite a few others. Installation is easy using `pip`. Issue the following from the command prompt:

     pip install filterpy
     
     
FilterPy is written by me, and the latest development version is always available at https://github.com/rlabbe/filterpy.
     
     

## Downloading and Running the Book

The book is stored in a github repository. From the command line type the following:

    git clone https://github.com/rlabbe/Kalman-and-Bayesian-Filters-in-Python.git
 
If you do not have git installed, browse to https://github.com/rlabbe/Kalman-and-Bayesian-Filters-in-Python where you can download the book via your browser.

Now, from the command prompt change to the directory that was just created, and then run Jupyter notebook:

    cd Kalman-and-Bayesian-Filters-in-Python
    juptyer notebook

A browser window should launch showing you all of the chapters in the book. Browse to the first chapter by clicking on it, then open the notebook in that subdirectory by clicking on the link.

More information about running the notebook can be found here:

http://jupyter-notebook-beginner-guide.readthedocs.org/en/latest/execute.html

## Using Juptyer Notebook

A complete tutorial on Jupyter Notebook is beyond the scope of this book. Many are available online. In short, Python code is placed in cells. These are prefaced with text like `In [1]:`, and the code itself is in a boxed area. If you press CTRL-ENTER while focus is inside the box the code will run and the results will be displayed below the box. Like this:

In [2]:
print(3+7.2)

10.2


If you have this open in Jupyter Notebook now, go ahead and modify that code by changing the expression inside the print statement and pressing CTRL+ENTER. The output should be changed to reflect what you typed in the code cell.

## SymPy

SymPy is a Python package for performing symbolic mathematics. The full scope of its abilities are beyond this book, but it can perform algebra, integrate and differentiate equations, find solutions to differential equations, and much more. For example, we use use it to compute the Jacobian of matrices and expected value integral computations.

First, a simple example. We will import SymPy, initialize its pretty print functionality (which will print equations using LaTeX). We will then declare a symbol for SymPy to use.

In [3]:
import sympy
sympy.init_printing(use_latex='mathjax')

phi, x = sympy.symbols('\phi, x')
phi

\phi

Notice how it prints the symbol `phi` using LaTeX. Now let's do some math. What is the derivative of $\sqrt{\phi}$?

In [4]:
sympy.diff('sqrt(phi)')

 1  
────
2⋅√φ

We can factor equations

In [5]:
sympy.factor(phi**3 -phi**2 + phi - 1)

           ⎛    2    ⎞
(\phi - 1)⋅⎝\phi  + 1⎠

and we can expand them.

In [6]:
((phi+1)*(phi-4)).expand()

    2             
\phi  - 3⋅\phi - 4

You can evauate an equation for specific values of its variables:

In [7]:
w =x**2 -3*x +4
print(w.subs(x, 4))
print(w.subs(x, 12))

8
112


You can also use strings for equations that use symbols that you have not defined:

In [8]:
x = sympy.expand('(t+1)*2')
x

2⋅t + 2

Now let's use SymPy to compute the Jacobian of a matrix. Given the function

$$h=\sqrt{(x^2 + z^2)}$$

find the Jacobian with respect to x, y, and z.

In [9]:
x, y, z = sympy.symbols('x y z')

H = sympy.Matrix([sympy.sqrt(x**2 + z**2)])

state = sympy.Matrix([x, y, z])
H.jacobian(state)

⎡     x                z      ⎤
⎢────────────  0  ────────────⎥
⎢   _________        _________⎥
⎢  ╱  2    2        ╱  2    2 ⎥
⎣╲╱  x  + z       ╲╱  x  + z  ⎦

Now let's compute the discrete process noise matrix $\mathbf Q$ given the continuous process noise matrix 
$$\mathbf Q = \Phi_s \begin{bmatrix}0&0&0\\0&0&0\\0&0&1\end{bmatrix}$$

The integral is 

$$\mathbf Q = \int_0^{\Delta t} \mathbf F(t)\mathbf Q\mathbf F^T(t)\, dt$$

where 
$$\mathbf F(\Delta t) = \begin{bmatrix}1 & \Delta t & {\Delta t}^2/2 \\ 0 & 1 & \Delta t\\ 0& 0& 1\end{bmatrix}$$

In [10]:
dt = sympy.symbols('\Delta{t}')
F_k = sympy.Matrix([[1, dt, dt**2/2],
                    [0,  1,      dt],
                    [0,  0,       1]])
Q = sympy.Matrix([[0,0,0],
                  [0,0,0],
                  [0,0,1]])

sympy.integrate(F_k*Q*F_k.T,(dt, 0, dt))

⎡         5           4           3⎤
⎢\Delta{t}   \Delta{t}   \Delta{t} ⎥
⎢──────────  ──────────  ──────────⎥
⎢    20          8           6     ⎥
⎢                                  ⎥
⎢         4           3           2⎥
⎢\Delta{t}   \Delta{t}   \Delta{t} ⎥
⎢──────────  ──────────  ──────────⎥
⎢    8           3           2     ⎥
⎢                                  ⎥
⎢         3           2            ⎥
⎢\Delta{t}   \Delta{t}             ⎥
⎢──────────  ──────────  \Delta{t} ⎥
⎣    6           2                 ⎦